In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
import selenium 
import sys
sys.path.append("../source")
import src

In [23]:
data_raw = pd.read_parquet("../data/reservas_hoteles.parquet")
df = pd.DataFrame(data_raw.copy())
type(df)

pandas.core.frame.DataFrame

In [24]:
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [5]:
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [6]:
df["ciudad"] = "Madrid"
df["ciudad"].nunique()

1

In [7]:
columnas_fechas = ["fecha_reserva", "inicio_estancia", "final_estancia"]

In [8]:
def to_date(columns, dataframe):
    for col in columns: 
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

In [9]:
df = to_date(columnas_fechas, df)

In [10]:
df["inicio_estancia"] = df["inicio_estancia"][0]
df["final_estancia"] = df["final_estancia"][0]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15098 non-null  datetime64[ns]
 8   final_estancia   15098 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [12]:
df.isnull().sum()

id_reserva            0
id_cliente            0
nombre                0
apellido              0
mail                  0
competencia           0
fecha_reserva      5172
inicio_estancia       0
final_estancia        0
id_hotel              0
precio_noche       5224
nombre_hotel          0
estrellas          5172
ciudad                0
dtype: int64

In [13]:
hoteles_agrupados = df.groupby("nombre_hotel")["id_hotel"].unique()

In [14]:
hoteles_agrupados = hoteles_agrupados[1:]
hoteles_agrupados

nombre_hotel
Gran Hotel Madrid         [47, 6, 13, 15, 35, 14, 49, 28, 48, 38, 40, 43...
Hotel Brisas del Mar      [38, 28, 36, 49, 29, 35, 6, 14, 47, 33, 27, 43...
Hotel Camino del Sol      [35, 47, 13, 15, 48, 40, 19, 49, 33, 28, 27, 3...
Hotel Costa Azul          [40, 48, 38, 13, 15, 19, 28, 49, 36, 27, 14, 2...
Hotel Encanto Real        [43, 28, 35, 14, 47, 6, 38, 33, 36, 19, 3, 15,...
Hotel Jardines del Rey    [3, 33, 2, 35, 47, 49, 15, 36, 29, 38, 43, 13,...
Hotel Las Estrellas       [29, 28, 40, 47, 6, 14, 49, 13, 36, 48, 35, 38...
Hotel Los Almendros       [28, 3, 35, 49, 43, 29, 36, 33, 47, 6, 14, 15,...
Hotel Luz de Madrid       [13, 2, 36, 15, 27, 47, 35, 33, 38, 40, 49, 14...
Hotel Maravilla Real      [40, 28, 49, 48, 14, 38, 29, 15, 6, 19, 2, 35,...
Hotel Mirador Real        [3, 14, 47, 15, 28, 43, 33, 36, 29, 49, 6, 2, ...
Hotel Monte Verde         [14, 29, 40, 13, 6, 15, 33, 43, 38, 36, 19, 28...
Hotel Palacio Imperial    [27, 36, 2, 13, 43, 40, 15, 19, 35, 14, 47, 28...

In [15]:
hoteles_df = pd.DataFrame(hoteles_agrupados)
hoteles_df["id_hotel"] = range(1, len(hoteles_df) + 1)
hoteles_df

,id_hotel
nombre_hotel,
Gran Hotel Madrid,1
Hotel Brisas del Mar,2
Hotel Camino del Sol,3
Hotel Costa Azul,4
Hotel Encanto Real,5
Hotel Jardines del Rey,6
Hotel Las Estrellas,7
Hotel Los Almendros,8
Hotel Luz de Madrid,9


In [19]:
merge_df = pd.merge(df, hoteles_df, on="nombre_hotel", how= "left")

In [20]:
merge_df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel_x,precio_noche,nombre_hotel,estrellas,ciudad,id_hotel_y
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid,NaN
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,12.0
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,2.0
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,3.0


In [ ]:
merge_df.drop(merge_df["id_hotel_x"])
merge_df.head(2)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel_x,precio_noche,nombre_hotel,estrellas,ciudad,id_hotel_y
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid,NaN


In [18]:
nombres_agrupados = pd.DataFrame(df.groupby(["nombre","apellido"])["mail"].apply(list).reset_index())
nombres_agrupados

,nombre,apellido,mail
0,Aarón,Alemán,[aarón.alemán@example.com]
1,Aarón,Aramburu,[aarón.aramburu@example.com]
2,Aarón,Arellano,[aarón.arellano@example.com]
3,Aarón,Azorin,[aarón.azorin@example.com]
4,Aarón,Barral,[aarón.barral@example.com]
...,...,...,...
14900,Úrsula,Morera,[úrsula.morera@example.com]
14901,Úrsula,Múgica,[úrsula.múgica@example.com]
14902,Úrsula,Sanchez,[úrsula.sanchez@example.com]
14903,Úrsula,Terrón,[úrsula.terrón@example.com]


In [19]:
df = pd.merge(df, hoteles_df[ "id_hotel"], on= "nombre_hotel", how= "left")
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel_x,precio_noche,nombre_hotel,estrellas,ciudad,id_hotel_y
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid,NaN
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,12.0
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,2.0
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,3.0


In [20]:
df.rename(columns= {"id_hotel_y" : "id_hotel_corregido"}, inplace=True)

In [21]:
df = pd.merge(df, nombres_agrupados[["nombre", "apellido", "mail"]], on=["nombre", "apellido"], how= "left")


In [22]:
df["id_hotel_corregido"]

0         NaN
1         NaN
2        12.0
3         2.0
4         3.0
         ... 
15093     7.0
15094     1.0
15095     6.0
15096    13.0
15097    14.0
Name: id_hotel_corregido, Length: 15098, dtype: float64

In [23]:
df["id_hotel_corregido"] = pd.to_numeric(df["id_hotel_corregido"], errors = "coerce").fillna(0).astype(int)
pd.DataFrame(df["id_hotel_corregido"])

,id_hotel_corregido
0,0
1,0
2,12
3,2
4,3
...,...
15093,7
15094,1
15095,6
15096,13


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_reserva          15098 non-null  object        
 1   id_cliente          15098 non-null  object        
 2   nombre              15098 non-null  object        
 3   apellido            15098 non-null  object        
 4   mail_x              15098 non-null  object        
 5   competencia         15098 non-null  bool          
 6   fecha_reserva       9926 non-null   datetime64[ns]
 7   inicio_estancia     15098 non-null  datetime64[ns]
 8   final_estancia      15098 non-null  datetime64[ns]
 9   id_hotel_x          15098 non-null  int64         
 10  precio_noche        9874 non-null   float64       
 11  nombre_hotel        15098 non-null  object        
 12  estrellas           9926 non-null   float64       
 13  ciudad              15098 non-null  object    

In [12]:
url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [17]:
src.scrap_hoteles(url)

NameError: name 'requests' is not defined